<a href="https://colab.research.google.com/github/Tiger-A/ai_integration_in_prod_examples/blob/main/integr_in_roduction_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Библиотеки
import os
import shutil

os.makedirs('templates', exist_ok=True)

os.makedirs('static', exist_ok=True)

os.makedirs('upload', exist_ok=True)

os.makedirs('ves', exist_ok=True)



!gdown --quiet 18_pLC7R8Yfnh_hPXAIZykJrN2oa0ch6N -O index.zip   # index
!unzip -q "index.zip" -d "index"

# Перемещаем файлы в соответствующие папки
shutil.move('index/favicon.ico', 'static/favicon.ico')
shutil.move('index/style.css', 'static/style.css')
shutil.move('index/index.html', 'templates/index.html')

!pip install supervision==0.2.0

# Ультралитикс
!pip install ultralytics
from IPython import display
display.clear_output()
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from PIL import Image
import imageio

from werkzeug.utils import secure_filename

from google.colab.patches import cv2_imshow
import numpy as np
import supervision as sv
import cv2
import pandas as pd

import matplotlib.pyplot as plt

import moviepy
from moviepy.editor import *
from moviepy.editor import VideoFileClip
from IPython.display import display, clear_output


import imageio
import moviepy.editor as mp


Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.1/78.2 GB disk)


In [2]:
#@title Функции


def predict_apple_foto():
    # Путь к папке upload
    upload_directory = '/content/upload'

    # Проверка наличия файла foto_ish.png
    if os.path.isfile(os.path.join(upload_directory, 'foto_ish.png')):
        frame = "/content/upload/foto_ish.png"
    # Проверка наличия файла foto_ish.jpg
    elif os.path.isfile(os.path.join(upload_directory, 'foto_ish.jpg')):
        frame = "/content/upload/foto_ish.jpg"

    # Load the YOLOv8 model
    yolo_model = YOLO('/content/ves/best_konveer_5_30.pt')


    # Папка с результатами
    output_directory = '/content/static'

        # Запускаем отслеживание YOLOv8 для кадра, сохраняя треки между кадрами
    results = yolo_model.predict(frame, conf=0.5)


    # Visualize the results
    im_bgr = results[0].plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

        # Show results to screen (in supported environments)
    results[0].show()

        # Save results to disk
    filename = os.path.join(output_directory, f'results.jpg')
    results[0].save(filename=filename)

    cls_apl = results[0].boxes.cls.tolist()

    count_ones = cls_apl.count(1.0)
    count_zeroes = cls_apl.count(0.0)
    return count_zeroes, count_ones


def predict_apple(kl_maska, xx1, yy1, xx2, yy2, xx3, yy3, xx4, yy4):

    video_path = '/content/upload/video_ish.mp4'
    #video_path = os.path.join('/content/upload/', uploaded_filename)
    print('video_path', video_path)

    frames_folder_poligon = '/content/itog' # сюда сложим несколько фоток-кадров для иллюстарации зоны внимания
    polygon = [(xx1, yy1), (xx2, yy2), (xx3, yy3), (xx4, yy4)] # кординаты зоны внимания ближе к центру
    hh = 1  # какие кадры берем для детекции (каждый hh)

    hhh=720
    polygon2 = [(0, 0), (xx2, 0), (xx2, yy2), (xx1, yy1), (xx1, hhh), (0, hhh)]


    # Load the YOLOv8 model
    yolo_model = YOLO('/content/ves/best_konveer_5_30.pt')


    # определяем размер видео
    video_info = sv.VideoInfo.from_video_path(video_path)
    #print(video_info.resolution_wh)

    cap = cv2.VideoCapture(video_path)

    # создаем папку
    os.makedirs(frames_folder_poligon, exist_ok=True)

    # счетчик кадров
    frame_count = 0

    # счетчик яблок
    apple_count = 0

    list_bb = []


    classes_list = ['fresh_apple', 'rotten_apple']
    class_labels_list = []





    while cap.isOpened():
        # Читаем кадр из видео
        success, frame = cap.read()

        frame_ishodnik = frame

        #print('\n_________________кадр: ', frame_count, '________________\n',)

        if not success:
            break

        # Берем каждый hh - кадр
        tt = frame_count % hh
        #print(tt)
        if tt == 0:
            # Запускаем отслеживание YOLOv8 для кадра, сохраняя треки между кадрами
            results = yolo_model.track(frame, persist=True)


            # забираем координаты ограничивающих рамок
            bboxes_ = results[0].boxes.xyxy.tolist()


           # забираем список id трекера для ограничивающих рамок
            #id_ = results[0].boxes.id.cpu().numpy().astype(int)
            try:
                id_ = results[0].boxes.id.cpu().numpy().astype(int)
            except AttributeError:
                #print("id_ не определено. Продолжаем выполнение кода.")
                continue  # Или другое действие, которое нужно выполнить в случае исключения




            # забираем метки классов
            class_labels_ = results[0].boxes.cls.cpu().numpy().astype(int)
            # забираем маски
            masks_ = results[0].masks.xyn


            #переводим координаты ограничивающих рамок в список координат ограничивающих рамок с целыми числами
            bboxes = list(map(lambda x: list(map(lambda y: int(y), x)), bboxes_))

            #print("bboxes = ", bboxes)

            # Если список не пустой
            if bboxes:
                for bbox_index, bbox in enumerate(bboxes): # берем по одной ограничивающей рамке (по одному лицу)
                    x1 = int(bbox[0])
                    y1 = int(bbox[1])
                    x2 = int(bbox[2])
                    y2 = int(bbox[3])

                    id_bb = id_[bbox_index] # id трекера текущей ограничивающей рамки
                    class_label = class_labels_[bbox_index] # метка класса текущей ограничивающей рамки
                    mask = masks_[bbox_index] # маска текущей ограничивающей рамки


                    #print("mask = ", mask)
                    class_name = classes_list[class_label] # получаем название класса по его метке



                # Отрисовка маски

                    # Преобразование координат маски в формат, ожидаемый функцией cv2.fillPoly()
                    mask_points = np.array(mask * np.array([frame.shape[1], frame.shape[0]]), np.int32)  # Без умножения на размер бокса и добавления его координат
                    mask_points = mask_points.reshape((-1, 1, 2))  # Преобразование формы для cv2.fillPoly()


                    # Координаты верхней средней точки ограничивающей рамки(макушка)
                    cx=int(x1+x2)//2
                    cy=int(y1)
                    w,h=x2-x1,y2-y1




                    # проверка вхождения верхней центральной точки ограничивающей рамки в полигон (зону интереса)
                    resultat= cv2.pointPolygonTest(np.array(polygon, np.int32), ((cx,cy)), False)

                     # проверка вхождения верхней центральной точки ограничивающей рамки в полигон (зону интереса)
                    resultat2= cv2.pointPolygonTest(np.array(polygon2, np.int32), ((cx,cy)), False)


                    if resultat<0 and resultat2<0:

                        #print('вне зоны интереса')

                        if id_bb in list_bb:
                            #print('вне зоны интереса')
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                            # Добавление подписи к боксу с ID класса
                            cv2.putText(frame, f"id{id_bb} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                            if kl_maska==1:
                                # Отрисовка контура маски
                                cv2.polylines(frame, [mask_points], True, (0, 255, 0), thickness=2)  # Цвет контура маски

                        else:
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

                            # Добавление подписи к боксу с ID класса
                            cv2.putText(frame, f"id{id_bb} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                            # Отрисовка контура маски
                            if kl_maska==1:
                                cv2.polylines(frame, [mask_points], True, (255, 0, 0), thickness=2)  # Цвет контура маски





                    if resultat<0 and resultat2>0:

                          #print('вне зоны интереса')
                          cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                          # Добавление подписи к боксу с ID класса
                          cv2.putText(frame, f"id{id_bb} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                          if kl_maska==1:
                              # Отрисовка контура маски
                              cv2.polylines(frame, [mask_points], True, (0, 255, 0), thickness=2)  # Цвет контура маски



                    if resultat>=0: #если resultat больше нуля то входит в полигон


                        if id_bb in list_bb:
                            #print("присутствует в списке.")
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                            # Добавление подписи к боксу с ID класса
                            cv2.putText(frame, f"id{id_bb} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


                            if kl_maska==1:
                                # Отрисовка контура маски
                                cv2.polylines(frame, [mask_points], True, (0, 255, 0), thickness=2)  # Цвет контура маски

                        else:
                            apple_count = apple_count + 1
                            #print("яблок: ", apple_count)
                            list_bb.append(id_bb)
                            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)

                            # Добавление подписи к боксу с ID класса
                            cv2.putText(frame, f"id{id_bb} {class_name}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


                            if kl_maska==1:
                                # Отрисовка контура маски
                                cv2.polylines(frame, [mask_points], True, (0, 255, 0), thickness=2)  # Цвет контура маски

                        #print(list_bb)

        # Добавление текста с количеством яблок на кадре
        cv2.putText(frame, f"Apple: {apple_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if frame_count < 10000:
            #Отрисовка полигона (зоны интереса) на 10 кадрах
            frame_filename2 = f'frame_{frame_count:04d}.jpg'
            frame_path2 = os.path.join(frames_folder_poligon, frame_filename2)

            cv2.polylines(frame,[np.array(polygon, np.int32)],True,(0,0,255),2)

            #cv2.polylines(frame,[np.array(polygon2, np.int32)],True,(255,0,0),2)


            #запись фото в папку
            cv2.imwrite(frame_path2, frame)

        frame_count += 1

    input_folder = '/content/itog'
    output_video_path = '/content/static/video_slow.mp4'  # имя полученного видео
    fps = 30  # Исходная частота кадров в видео
    target_fps = 15  # Желаемая частота кадров (можно выбрать любую)

    # Получаем список файлов в папке
    frame_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

    # Сортируем файлы по имени, чтобы они шли в правильном порядке
    frame_files.sort()

    # Определяем размер кадра из первого изображения
    frame_kadr = cv2.imread(os.path.join(input_folder, frame_files[0]))
    height, width, _ = frame_kadr.shape

    # Создаем объект VideoWriter с новой частотой кадров и новым размером кадра
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Выбираем кодек для MP4
    out = cv2.VideoWriter(output_video_path, fourcc, target_fps, (800, 450))  # измененный размер кадра

    # Читаем каждый кадр изображения и записываем его в видео с уменьшенной частотой кадров
    for file_name in frame_files:
        img_path = os.path.join(input_folder, file_name)
        frame_kadr = cv2.imread(img_path)
        resized_frame = cv2.resize(frame_kadr, (800, 450))  # изменяем размер каждого кадра
        out.write(resized_frame)

    # Закрываем объект VideoWriter
    out.release()

# Путь к видеофайлу
    video_path_3 = '/content/static/video_slow.mp4'

        # Путь и имя создаваемого GIF файла
    output_gif_path = '/content/static/animated.gif'

        # Загружаем видео
    clip = mp.VideoFileClip(video_path_3)

        # Изменяем размер видео
    clip_resized = clip.resize(width=500, height=280)

        # Создаем GIF из видео
    clip_resized.write_gif(output_gif_path, fps=10, loop=0)

    return apple_count


def upload_ves2():
    print("Идет загрузка весов")
    # Путь к папке upload
    upload_directory = 'ves'

    if 'file' not in request.files:
        return 'No file part'

    file = request.files['file']

    if file.filename == '':
        return 'No selected file'

    if file:

        file.save('ves/'+ file.filename)




def upload_file2():
    print("Идет загрузка файла")
    # Путь к папке upload
    upload_directory = 'upload'

    # Удаление всех файлов в папке upload
    for filename in os.listdir(upload_directory):
        file_path = os.path.join(upload_directory, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(e)

    if 'file' not in request.files:
        return 'No file part'

    file = request.files['file']

    if file.filename == '':
        return 'No selected file'

    if file:
        # Получение расширения файла
        file_extension = os.path.splitext(file.filename)[1].lower()

        # Определение нового имени файла в зависимости от расширения
        if file_extension == '.png':
            new_filename = 'foto_ish.png'
        elif file_extension == '.jpg':
            new_filename = 'foto_ish.jpg'
        elif file_extension == '.mp4':
            new_filename = 'video_ish.mp4'
        else:
            return 'Unsupported file type'

        # Сохранение файла с новым именем
        filename = secure_filename(new_filename)
        file.save(os.path.join('upload', filename))



def present_file2():

    # Путь к папке upload
    upload_directory = '/content/upload'

    files = os.listdir(upload_directory)


        # Определение нового имени файла в зависимости от расширения
    if files[0] == 'foto_ish.png':
        new_filename = 'foto_ish.png'
        # Копирование файла
        shutil.copy('/content/upload/foto_ish.png', '/content/static/ish_png.png')
        file_i = "foto_png"
        return file_i

                # Определение нового имени файла в зависимости от расширения
    elif files[0] == 'foto_ish.jpg':
        new_filename = 'foto_ish.jpg'
        # Копирование файла
        shutil.copy('/content/upload/foto_ish.jpg', '/content/static/ish_jpg.jpg')
        file_i = "foto_jpg"
        return file_i


    elif files[0] == 'video_ish.mp4':

        # Путь к видеофайлу
        video_path = '/content/upload/video_ish.mp4'

        # Путь и имя создаваемого GIF файла
        output_gif_path = '/content/static/ish_animated.gif'

        # Загружаем видео
        clip = mp.VideoFileClip(video_path)

        # Изменяем размер видео
        clip_resized = clip.resize(width=500, height=280)

        # Создаем GIF из видео
        clip_resized.write_gif(output_gif_path, fps=10, loop=0)

        file_i = "video_mp4"
        return file_i

Ссылка на приложение

In [3]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://6uwyrt7xm1i-496ff2e9c6d22116-5000-colab.googleusercontent.com/


Приложение на Flask

In [4]:
from flask import Flask, render_template, request, send_file


app = Flask(__name__, template_folder = '/content/templates')

# Загрузка первой странички

@app.route("/")
def home():
  return render_template("index.html")

# Загрузка весов

@app.route('/upload_ves', methods=['POST'])
def upload_ves():
    upload_ves2()  # Функция загрузки весов
    return render_template("index.html", text_v="Веса загружены!")

# Загрузка файла

@app.route('/upload', methods=['POST'])
def upload_file():
    upload_file2() # Функция загрузки файла
    return render_template("index.html", text_z="Файл загружен!")

# Отображение загруженного файла

@app.route('/present', methods=['POST'])
def present_file():
    file_i = present_file2()  # Функция отображения файла

    if file_i == 'foto_png':
        return render_template("index.html", foto_png="Ok")

    elif file_i == 'foto_jpg':
        return render_template("index.html", foto_jpg="Ok")

    elif file_i == 'video_mp4':
        return render_template("index.html", video_mp4="Ok")

# Предсказание на видео

@app.route("/predict", methods=['POST'])
def predict():

    checkbox1 = request.form.get('checkbox1')  # Получаем значение первого флажка

    x1 = request.form.get('x1')  # Получаем значение x1
    y1 = request.form.get('y1')  # Получаем значение y1
    x2 = request.form.get('x2')  # Получаем значение x2
    y2 = request.form.get('y2')  # Получаем значение y2
    x3 = request.form.get('x3')  # Получаем значение x3
    y3 = request.form.get('y3')  # Получаем значение y3
    x4 = request.form.get('x4')  # Получаем значение x4
    y4 = request.form.get('y4')  # Получаем значение y4

    if x1 == "":  # если не ввели значения, то принимаем по умолчанию
        x1 = 20
        y1 = 540
        x2=940
        y2 = 420
        x3=1280
        y3 = 470
        x4=220
        y4 = 670

    if checkbox1 == 'on':  # если чекбокс включен, то отображаем маску
        kl_maska = 1
    else:
        kl_maska = 0
    print("Отображать маску:", kl_maska)

    apple_count = predict_apple(kl_maska, x1, y1, x2, y2, x3, y3, x4, y4) # Функция предсказания на видео

    return render_template("index.html", apple_index = apple_count)

# Предсказание на фото

@app.route("/predict_foto", methods=['POST'])
def predict_foto():

    apple_fresh, apple_rotten = predict_apple_foto()  # Функция предсказания на фото

    return render_template("index.html", apple_fresh = apple_fresh, apple_rotten = apple_rotten)

# Показать видео

@app.route("/video", methods=['POST'])
def frames_to_video():

    return render_template("index.html", video_ok = "ok")

# Скачивание файла

@app.route('/download')
def download_file():
    # Путь к файлу, который вы хотите скачать
    file_path3 = '/content/static/video_slow.mp4'
    # Отправка файла для скачивания
    return send_file(file_path3, as_attachment=True)

if __name__ == "__main__":
  app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:11:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:11:30] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:11:31] "GET /static/favicon.ico HTTP/1.1" 200 -
ERROR:__main__:Exception on /present [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py"

Идет загрузка файла
Идет загрузка файла


INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:03] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:05] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:06] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:08] "POST /present HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:09] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Apr/2024 18:13:09] "GET /static/ish_jpg.jpg HTTP/1.1" 200 -
ERROR:__main__:Exception on /predict_foto [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
